In [ ]:
import proplot as pplt
import numpy as np
from ruamel.yaml import YAML
import xarray as xr

from pathlib import Path

In [ ]:
plotting_dir = Path("plots/ensemble/ec-earth-3")
plotting_dir.mkdir(exist_ok=True, parents=True)
ensemble_directory = Path("PAPA/ensemble_output")

In [ ]:
schwarz_directories = []
yaml = YAML(typ="unsafe", pure=True)
for date_dir in ensemble_directory.glob("*"):
    if date_dir.is_dir():
        schwarz_directories.append(date_dir / "schwarz")
experiments = []
non_converged_counter = 0
for schwarz_dir in schwarz_directories:
    with open(schwarz_dir / "setup_dict.yaml") as yaml_file:
        experiment = yaml.load(yaml_file)
        converged = experiment.iterate_converged["inf-norm"]
        if converged:
            experiments.append(experiment)
        else:
            non_converged_counter += 1

iterations = np.array([experiment.iteration for experiment in experiments])
# start_dates = np.array([schwarz_dict["run_start_date"] for schwarz_dict in schwarz_dicts])
iterations = xr.DataArray(iterations, name="iterations")

In [ ]:
print(f"Not converged: {non_converged_counter}")

In [ ]:
iteration_count = iterations.groupby(iterations).count()
iteration_count.name = "Iteration Count"

In [ ]:
fig, ax = pplt.subplots(width="40em", height="20em")
ax.bar(iteration_count, absolute_width=True, color="k")
ax.format(
    xlabel="Number of Iterations until Termination Criterion Satisfied",
    ylabel="Number of Experiments",
    xlim=[0, 31],
    xticks=np.arange(2, 31, 2),
    xtickminor=True,
    xminorticks=np.arange(0, 31),
)
fig.savefig(plotting_dir / "swr_iteration_count.pdf")

In [ ]:
iterations.mean()

In [ ]:
iterations.median()